<a href="https://colab.research.google.com/github/matsu323/LLM-submissson/blob/main/LLMcharenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
!pip install rdkit-pypi
!pip install openai
!pip install pubchempy
import pubchempy as pcp

!pip install rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=c109b2ad6a722b7331637501f95e7efd658e1038b39cd453cff31b89c1bc65f9
  Stored in directory: 

In [3]:
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity

In [4]:
sdf_file = '/content/CBLB_inhibitors_vsF.sdf'
suppl = Chem.SDMolSupplier(sdf_file)


# 分子をSMILES形式に変換する
molecules = []
for mol in suppl:
    if mol is not None:
        smiles = Chem.MolToSmiles(mol)
        molecules.append(smiles)
activity_scores=[]


In [5]:
#PubChemに登録されていないもの
no=[4,
20,
21,
45,
51,
52,
61,
64,
79,
109,
127,
132,
134,
167,
178,
184,
185,
256,
385,
404,
406,
408,
409,
417,
420,
423,
425,
469,
518,
519,
570,
571,
587,
588,
592,
594,
597,
599,
602,
613,
722,
819,
820,
823,
825,
831,
832,
834]
remol = [molecules[i] for i in range(len(molecules)) if i not in no]


In [9]:
num=30

In [6]:
#遠いもの探索
index=[461]
num=30
for i in range(num):
  min=[0,1]
  for j in range(len(remol)):
    tmp=0
    fp1=AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(remol[j]), 2, nBits=1024)
    for k in range(len(index)):
      fp2=AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(remol[index[k]]), 2, nBits=1024)
      sim=TanimotoSimilarity(fp1, fp2)
      if sim>tmp:
        tmp=sim
    if min[1]>tmp:
      min=[j,tmp]
  index.append(min[0]) 
index=sorted(index)


In [10]:
print(index)
# print(len(remol))
smiles=[]
properties = ['IUPACName', 'CanonicalSMILES']
for i in range(num):
  smiles.append(pcp.get_properties(properties,remol[index[i]],'smiles'))
l=[]
for i in range(num):
  l.append([smiles[i][0]['CanonicalSMILES'],smiles[i][0]['IUPACName']])
for i in range(num):
  print(l[i][0])

[12, 44, 52, 54, 71, 73, 83, 172, 338, 348, 424, 442, 457, 461, 472, 507, 568, 576, 583, 619, 628, 634, 638, 639, 653, 702, 764, 809, 828, 829, 844]
CC(C1=CC(=CC=C1)N2N=CC(=N2)C3=CC(=CC=C3)OC)SC4=NN=CN4C
CC(C1=CC=CC(=C1)C2=CC(=NO2)C3=CC=C(C=C3)C(=O)NC)SC4=NN=CN4C
CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=CC(=C4)C5CCC5
CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=C(C=C4N5CCC(C5)O)C(F)(F)F
CC(C1=CC(=CC=C1)NC(=O)N2CC3(CNC3)OC4=C2N=CC=C4)(C(C5=NN=CN5C)(F)F)F
CC(CC1=NN=CN1C)C2=CC(=CC=C2)N3CC4=C(C3=O)C=CC=C4Br
CN1C=NN=C1C2CC2C3=CC(=CC=C3)N4CC5=C(C4=O)C=CN=C5C(F)(F)F
CC1CN(CCN1C2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C(C)(C(C5=NN=CN5C)(F)F)F)C(=C2)C(F)(F)F)C
CC(C1=CC2=C(CN(C2=O)C3=CC=CC(=C3)C4(COC4)C(C5=NN=CN5C)F)C(=C1)C(F)(F)F)N6CCC7(C6)CC7
CCOC1=CC(=CC(=C1)N2CC3=C(C2=O)C=C(C=C3C(F)(F)F)CN4CCC(C4)F)C5(COC5)CC6=NN=CN6C
CC1CN(CCO1)CC2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C5=C(C=NN5C)C6=NN=CN6C)C(=C2)C(F)(F)F
CC1CC(CN1C(=O)C2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C(C)CC5=C(C=NN5)C)C(=C2)C(F)(F)F)O
CC(C1=CC(=CC=C1)N2CC3=C(C2=O)C=C(C=

In [40]:
slist=[
'CC(C1=CC=CC(=C1)C2=CC(=NO2)C3=CC=C(C=C3)C(=O)NC)SC4=NN=CN4N',
'CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=C(C=C4N5CCC(C5)O)C(F)(F)Cl',
'CC(CC1=NN=CN1C)C2=CC(=CC=C2)N3CC4=C(C3=O)C=CC=C4I',
'CC(C1=CC2=C(CN(C2=O)C3=CC=CC(=C3)C4(COC4)C(C5=NN=CN5C)F)C(=C1)C(F)(F)Cl)N6CCC7(C6)CC7',
'CCOC1=CC(=CC(=C1)N2CC3=C(C2=O)C=C(C=C3C(F)(F)Cl)CN4CCC(C4)F)C5(COC5)CC6=NN=CN6C',
'CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=C(C=C4N5CCC(C5)O)C(F)(F)I',
'CC(C1=CC2=C(CN(C2=O)C3=CC=CC(=C3)C4(COC4)C(C5=NN=CN5C)F)C(=C1)C(F)(F)Br)N6CCC7(C6)CC7',
'CCOC1=CC(=CC(=C1)N2CC3=C(C2=O)C=C(C=C3C(F)(F)Br)CN4CCC(C4)F)C5(COC5)CC6=NN=CN6C',
'CC(C1=CC2=C(CN(C2=O)C3=CC=CC(=C3)C4(COC4)C(C5=NN=CN5C)F)C(=C1)C(F)(F)Br)N6CCC7(C6)CC7',
'CCOC1=CC(=CC(=C1)N2CC3=C(C2=O)C=C(C=C3C(F)(F)Br)CN4CCC(C4)F)C5(COC5)CC6=NN=CN6C',
'CC1CN(CCO1)CC2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C5=C(C=NN5C)C6=NN=CN6Cl)C(=C2)C(F)(F)F',
'CC1CC(CN1C(=O)C2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C(C)CC5=C(C=NN5)C)C(=C2)C(F)(F)Cl)O',
'CC(C1=CC(=CC=C1)N2CC3=C(C2=O)C=C(C=C3)C(C)(C)O)SC4=NN=CN4Cl',
'CCNC1=NC(=CC(=C1)C(C)SC2=NN=CN2C)N3CC4=C(C3=O)C=CC=C4C(F)(F)Cl',
'CC(CC1=NN=CN1C)C2=CC(=CC=C2)C(=O)NC3=CC=CC(=N3)C(F)(F)Cl',
'CC(C)(CC1=NN=CN1C)C2=CC(=CC=C2)NC(=O)C3=NC(=NC=C3)C(F)(F)Cl',
'CC(CC1=NN=CN1C)C2=CC(=CC=C2)NC(=O)C3=NC=CC(=C3)C(=O)N4CCC5(C4)COC5Cl',
'CCOC1=CC(=CC(=N1)NC(=O)C2=NC(=NC(=C2)C)C3CC3)C(C)(C(C4=NN=CN4C)(F)F)Cl',
'CC1=CC(=NC(=N1)C2CC2)C(=O)NC3=CC=CC(=C3)C(C)CC4=NN=CS4Cl',
'CN1C=NN=C1C2CC2C3=CC(=CC=C3)NC(=O)C4=NC5=C(C=CC=C5N4)C(F)(F)Cl',
'CC1=CON=C1CC2(COC2)C3=CC(=CC=C3)NC(=O)C4=NC(=NC(=C4)CN5CCC6(C5)CC6)C7CC7Cl',
'CN1C=NN=C1CC2(COC2)C3=CC(=CC=C3)N4CC5C(CCN5C4=O)C(F)(F)Cl',
'CN1C=NN=C1CC2(COC2)C3=CC(=CC=C3)NC(=O)C4=C5C(=NN4)CCCO5Cl',
'CC1=CN=NN1CC(C)C2=CC(=CC=C2)NC(=O)C3=NC(=CC=C3)C(F)(F)Cl',
'CCN1C=C(C=N1)CN(C)C(=O)NC2=CC=CC(=C2)C(C)SC3=NN=CN3Cl',
'CC(C1=CC(=CC=C1)NC(=O)N2CCOC(C2)C3=NOC=N3)SC4=NN=CN4Cl',
'CC(C1=CC(=C(C=C1)OC(F)(F)F)NC(=O)C2=NC(=CC=C2)C(F)(F)Cl)SC3=NN=CN3C',
'CN1C=NN=C1SC(CC=C)C2=CC(=CC=C2)NC(=O)C3=NC4=CC=CC=C4C=C3Cl',
'CC(C1=CC(=NC=C1)NC(=O)C2=NC=C3CCCCC3=C2)SC4=NN=CN4Cl',
'CC(C1=CC(=CC=C1)NC(=O)C2=CC3=CC=CC=C3C=N2)SC4=NC=NN4Cl'
]
valid=[]
for i in range(len(slist)):
  try:
     mol= Chem.MolFromSmiles(slist[i])
     if mol is None :
      print(i)
     else:
      valid.append(slist[i])
  except:
    print(i)
print(len(valid))

22
29


[11:09:33] Explicit valence for atom # 28 O, 3, is greater than permitted


In [43]:
#もと900の谷本との比較
max=[]
for i in range(len(valid)):
  mol=Chem.MolFromSmiles(valid[i])
  m=[0,0,i]
  for j in range(len(remol)):
    mol1=Chem.MolFromSmiles(remol[j])
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=1024)
    similarity = TanimotoSimilarity(fp1, fp2)
    if m[0]< similarity:
      m[0]=similarity
      m[1]=j
  max.append(m)




[[0.6764705882352942, 499, 18], [0.6966292134831461, 442, 11], [0.6986301369863014, 619, 19], [0.6987951807228916, 54, 5], [0.7073170731707317, 54, 1], [0.7258064516129032, 822, 28], [0.775, 568, 16], [0.7857142857142857, 809, 26], [0.7931034482758621, 73, 2], [0.8387096774193549, 44, 0], [0.8412698412698413, 457, 12], [0.8484848484848485, 653, 23], [0.8524590163934426, 828, 27], [0.8536585365853658, 628, 20], [0.8571428571428571, 702, 24], [0.864406779661017, 472, 14], [0.8771929824561403, 639, 22], [0.8787878787878788, 634, 21], [0.8840579710144928, 576, 17], [0.8852459016393442, 507, 15], [0.8888888888888888, 764, 25], [0.891566265060241, 424, 10], [0.9, 461, 13], [0.9012345679012346, 338, 3], [0.9036144578313253, 348, 4], [0.9125, 338, 6], [0.9125, 338, 8], [0.9146341463414634, 348, 7], [0.9146341463414634, 348, 9]]


In [42]:
# 提案化合物同士がかぶってるか
print(len(valid))
import copy
validunique=copy.deepcopy(valid)
flag=[]
for i in range(len(valid)):
  for j in range(i+1,len(valid)):
    mol1=Chem.MolFromSmiles(valid[i])
    mol2=Chem.MolFromSmiles(valid[j])
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=1024)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, 2, nBits=1024)
    similarity = TanimotoSimilarity(fp1, fp2)
    if similarity==1:
      validunique.remove(valid[i])
      print(1)

print(len(validunique))

29
1
1
27


In [51]:
max=[]
for i in range(len(validunique)):
  mol=Chem.MolFromSmiles(validunique[i])
  m=[0,0,i]
  for j in range(len(remol)):
    mol1=Chem.MolFromSmiles(remol[j])
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol1, 2, nBits=1024)
    similarity = TanimotoSimilarity(fp1, fp2)
    if m[0]< similarity:
      m[0]=similarity
      m[1]=j
  max.append(m)
print(len(max))
print(sorted(max))
# for i in range(9):
#   print(max[i])

27
[[0.6764705882352942, 499, 16], [0.6966292134831461, 442, 9], [0.6986301369863014, 619, 17], [0.6987951807228916, 54, 5], [0.7073170731707317, 54, 1], [0.7258064516129032, 822, 26], [0.775, 568, 14], [0.7857142857142857, 809, 24], [0.7931034482758621, 73, 2], [0.8387096774193549, 44, 0], [0.8412698412698413, 457, 10], [0.8484848484848485, 653, 21], [0.8524590163934426, 828, 25], [0.8536585365853658, 628, 18], [0.8571428571428571, 702, 22], [0.864406779661017, 472, 12], [0.8771929824561403, 639, 20], [0.8787878787878788, 634, 19], [0.8840579710144928, 576, 15], [0.8852459016393442, 507, 13], [0.8888888888888888, 764, 23], [0.891566265060241, 424, 8], [0.9, 461, 11], [0.9012345679012346, 338, 3], [0.9036144578313253, 348, 4], [0.9125, 338, 6], [0.9146341463414634, 348, 7]]


In [55]:
sormax=sorted(max)
for i in range(10):
  print(validunique[sormax[i][2]])

CC1=CC(=NC(=N1)C2CC2)C(=O)NC3=CC=CC(=C3)C(C)CC4=NN=CS4Cl
CC1CC(CN1C(=O)C2=CC3=C(CN(C3=O)C4=CC=CC(=C4)C(C)CC5=C(C=NN5)C)C(=C2)C(F)(F)Cl)O
CN1C=NN=C1C2CC2C3=CC(=CC=C3)NC(=O)C4=NC5=C(C=CC=C5N4)C(F)(F)Cl
CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=C(C=C4N5CCC(C5)O)C(F)(F)I
CC(CC1=NN=CN1C)C2=CC=CC(=C2)C3=NNC4=C3N=C(C=C4N5CCC(C5)O)C(F)(F)Cl
CC(C1=CC(=CC=C1)NC(=O)C2=CC3=CC=CC=C3C=N2)SC4=NC=NN4Cl
CC(CC1=NN=CN1C)C2=CC(=CC=C2)NC(=O)C3=NC=CC(=C3)C(=O)N4CCC5(C4)COC5Cl
CN1C=NN=C1SC(CC=C)C2=CC(=CC=C2)NC(=O)C3=NC4=CC=CC=C4C=C3Cl
CC(CC1=NN=CN1C)C2=CC(=CC=C2)N3CC4=C(C3=O)C=CC=C4I
CC(C1=CC=CC(=C1)C2=CC(=NO2)C3=CC=C(C=C3)C(=O)NC)SC4=NN=CN4N
